In [1]:
%reset
import numpy as np
import pandas as pd
import os
%pwd

'd:\\다운로드\\Python\\competition\\global-wheat-detection\\working'

In [2]:
train = pd.read_csv('../input/global-wheat-detection/train.csv')
submission = pd.read_csv('../input/global-wheat-detection/sample_submission.csv')

In [3]:
train_bbox = train['bbox'].str.lstrip('[').str.rstrip(']').str.split(', ', expand=True).astype(float)
train_bbox.columns = ['x0', 'y0', 'xl', 'yl']
train_bbox['class'] = 0
train_bbox['xc'] = (train_bbox['x0'] + train_bbox['xl']/2) / train['width']
train_bbox['yc'] = (train_bbox['y0'] + train_bbox['yl']/2) / train['height']
train_bbox['xln'] = train_bbox['xl'] / train['width']
train_bbox['yln'] = train_bbox['yl'] / train['height']
train_bbox['image_id'] = train['image_id']
train_bbox.drop(columns=train_bbox.columns[0:4], inplace=True)
train_bbox

,class,xc,yc,xln,yln,image_id
0,0,0.841797,0.234375,0.054688,0.035156,b6ab77fd7
1,0,0.284180,0.563477,0.126953,0.056641,b6ab77fd7
2,0,0.404297,0.570312,0.072266,0.156250,b6ab77fd7
3,0,0.867676,0.145020,0.106445,0.104492,b6ab77fd7
4,0,0.085938,0.197754,0.121094,0.114258,b6ab77fd7
...,...,...,...,...,...,...
147788,0,0.103516,0.650879,0.082031,0.092773,5e0747034
147789,0,0.337402,0.576172,0.104492,0.080078,5e0747034
147790,0,0.199707,0.257324,0.137695,0.069336,5e0747034
147791,0,0.509766,0.051270,0.179688,0.077148,5e0747034


In [4]:
train_imgid = pd.Series(data=train_bbox['image_id'].drop_duplicates())
train_imgid_y = pd.Series(data=None, index=train_imgid.index, dtype=int)

In [5]:
from sklearn.model_selection import train_test_split
train_x_imgid, valid_x_imgid, _, _ = train_test_split(train_imgid, train_imgid_y, test_size=0.3, random_state=0)

In [6]:
import shutil
def clean_dir(path_dir):
    if os.path.exists(path_dir):
        shutil.rmtree(path_dir)
    os.mkdir(path_dir)

In [7]:
TRAIN_ORG = '../input/global-wheat-detection/train/'
TEST_ORG = '../input/global-wheat-detection/test/'
SPLIT = '../train-valid-split/'
IMG_SPL = SPLIT + 'images/'
LBL_SPL = SPLIT + 'labels/'
TRAIN_IMG_SPL = IMG_SPL + 'train/'
VALID_IMG_SPL = IMG_SPL + 'valid/'
TRAIN_LBL_SPL = LBL_SPL + 'train/'
VALID_LBL_SPL = LBL_SPL + 'valid/'

In [8]:
clean_dir(SPLIT)
clean_dir(IMG_SPL)
clean_dir(LBL_SPL)
clean_dir(TRAIN_IMG_SPL)
clean_dir(VALID_IMG_SPL)
clean_dir(TRAIN_LBL_SPL)
clean_dir(VALID_LBL_SPL)
for i, name in enumerate(train_x_imgid):
    shutil.copy(TRAIN_ORG + name + '.jpg', TRAIN_IMG_SPL)
    train_bbox[train_bbox['image_id'] == name].iloc[:, 0:5].to_csv(TRAIN_LBL_SPL + name + '.txt', mode='a', sep=' ', header=None, index=None)
    print(str(i+1) + '/' + str(train_x_imgid.shape[0]), end='\r')
for i, name in enumerate(valid_x_imgid):
    shutil.copy(TRAIN_ORG + name + '.jpg', VALID_IMG_SPL)
    train_bbox[train_bbox['image_id'] == name].iloc[:, 0:5].to_csv(VALID_LBL_SPL + name + '.txt', mode='a', sep=' ', header=None, index=None)
    print(str(i+1) + '/' + str(valid_x_imgid.shape[0]), end='\r')

1012/1012